## Лаб.11 Теммодель

### Тематическое моделирование текстов

---

#### Входные данные: 
датасет в формате `json`


**Порядок выполнения работы:**
1. Импортируйте файл json:<br>`import json`<br>`with open("recipes.json") as f:`<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`recipes = json.load(f)`
2. Выведите на экран несколько рецептов по их номерам.
3. Для подготовки данных `json` необходимо составить словарь из слов поля `ingredients`, а также построить корпус, содержащий число вхождений для каждого слова из словаря.<br><u>Пояснения</u>. Словарь содержит элементы списка из поля `ingredients`:<br>`romaine lettuce`, `black olives`, `grape tomatoes`, `garlic`, `pepper` и т.д.<br>Корпус представляет собой список пар значений: индекс слова в словаре, число вхождений:<br>`[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1),...]`<br>О создании словаря в `Gensim`: [Общее описание](https://www.tutorialspoint.com/gensim/gensim_creating_a_dictionary.htm), [документация класса](https://radimrehurek.com/gensim/corpora/dictionary.html).<br>Создание корпуса – путем применения метода `doc2bow()` к созданному словарю (см. документацию).
4. Выведите размер словаря.
5. Зафиксируйте `random.seed`
6. Обучите модель `LDA` с параматрами: 40 тем, количество проходов по коллекции 5, остальные параметры по умолчанию. Сохраните модель в отдельный файл (см. [документацию класса `LdaModel`](https://radimrehurek.com/gensim/models/ldamodel.html)).
7. Реализуйте представления сформированных моделью тем, используя методы `show_topics()`, `show_topic()`, `get_topic_terms()`, `get_topics()`.
8. Попытайтесь интерпретировать сформированные темы. Например, сопоставьте темы с номерами 0, 1, 4 и 5.
9. Для нескольких слов определите наиболее релевантные этим словам темы (метод `get_term_topics()`).
10. Оцените качество полученной модели, вычислив когерентность токенов и перплексию.

---

In [ ]:
!pip install pyldavis

In [ ]:
import json
import gensim
import random
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models
from collections import Counter
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings('ignore')

1. Импортируйте файл json:<br>`import json`<br>`with open("recipes.json") as f:`<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`recipes = json.load(f)`

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [4]:
import json

with open("gdrive/My Drive/Colab Notebooks/data/recipes.json") as f:
    recipes = json.load(f)

---

2. Выведите на экран несколько рецептов по их номерам.

In [5]:
# display multiple recipes by their numbers.

print(recipes[0])
print(recipes[1])
print(recipes[2])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}
{'id': 25693, 'cuisine': 'southern_us', 'ingredients': ['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']}
{'id': 20130, 'cuisine': 'filipino', 'ingredients': ['eggs', 'pepper', 'salt', 'mayonaise', 'cooking oil', 'green chilies', 'grilled chicken breasts', 'garlic powder', 'yellow onion', 'soy sauce', 'butter', 'chicken livers']}


In [6]:
import pandas as pd

df = pd.io.json.json_normalize(recipes)

In [7]:
df.cuisine.unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

In [8]:
df.loc[df['cuisine'] == 'russian']

id  cuisine                                        ingredients
369    38346  russian            [water, grits, mozzarella cheese, salt]
430    35962  russian  [water, dill, sugar, buttermilk, onions, red b...
669    49388  russian  [black pepper, salt, white sandwich bread, who...
774    15753  russian  [white pepper, salt, eggs, unsalted butter, br...
778    39356  russian  [white cabbage, salt, extra-virgin olive oil, ...
...      ...      ...                                                ...
39296   4486  russian  [granulated sugar, all-purpose flour, brandy, ...
39368   3878  russian  [bread crumb fresh, vegetable oil, sour cream,...
39493  31866  russian  [kefir, vegetable oil, eggs, baking soda, salt...
39653  18039  russian  [hazelnuts, baking soda, salt, confectioners s...
39719  34814  russian  [pepper, garlic, pork sirloin, onions, olive o...

[489 rows x 3 columns]

In [9]:
recipes[929]

{'id': 21896,
 'cuisine': 'russian',
 'ingredients': ['mayonaise',
  'salt',
  'turkey hot dogs',
  'green onions',
  'potatoes',
  'dill pickles',
  'eggs',
  'peas']}

---

3. Для подготовки данных `json` необходимо составить словарь из слов поля `ingredients`, а также построить корпус, содержащий число вхождений для каждого слова из словаря.<br><u>Пояснения</u>. Словарь содержит элементы списка из поля `ingredients`:<br>`romaine lettuce`, `black olives`, `grape tomatoes`, `garlic`, `pepper` и т.д.<br>Корпус представляет собой список пар значений: индекс слова в словаре, число вхождений:<br>`[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1),...]`<br>О создании словаря в `Gensim`: [Общее описание](https://www.tutorialspoint.com/gensim/gensim_creating_a_dictionary.htm), [документация класса](https://radimrehurek.com/gensim/corpora/dictionary.html).<br>Создание корпуса – путем применения метода `doc2bow()` к созданному словарю (см. документацию).

In [10]:
# to prepare the json data, you need to build a dictionary from the words of the 'ingredients' field, 
# and also build a corpus containing the number of occurrences for each word from the dictionary

dictionary = gensim.corpora.Dictionary([recipe['ingredients'] for recipe in recipes])
corpus = [dictionary.doc2bow(recipe['ingredients']) for recipe in recipes] # convert document (a list of words) into the bag-of-words format

---

4. Выведите размер словаря.

In [11]:
# print the size of the dictionary.

print(len(dictionary))

6714


---

5. Зафиксируйте `random.seed`

In [12]:
# fix random.seed

random.seed(42)

---

6. Обучите модель `LDA` с параматрами: 40 тем, количество проходов по коллекции 5, остальные параметры по умолчанию. Сохраните модель в отдельный файл (см. [документацию класса `LdaModel`](https://radimrehurek.com/gensim/models/ldamodel.html)).

In [13]:
# train the LDA model using pyLDAvis with parameters: 40 topics, number of passes through 
# the collection 5, other parameters are default, save the model to a separate file

lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=40, passes=5)
vis = pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=corpus, dictionary=dictionary)
pyLDAvis.save_html(vis, "gdrive/My Drive/Colab Notebooks/save_viz.html")
pyLDAvis.display(vis)

---

7. Реализуйте представления сформированных моделью тем, используя методы `show_topics()`, `show_topic()`, `get_topic_terms()`, `get_topics()`.

In [14]:
# implement views of the topics generated by the model using the show_topics(), show_topic(), get_topic_terms(), get_topics() methods

from pprint import pprint

pprint(lda.show_topics())
print()
pprint(lda.show_topic(0))
print()
pprint(lda.get_topic_terms(0))
print()
pprint(lda.get_topics())

# details: https://radimrehurek.com/gensim/models/ldamodel.html

[(39,
  '0.058*"extra-virgin olive oil" + 0.057*"purple onion" + 0.054*"salt" + '
  '0.054*"red wine vinegar" + 0.054*"olive oil" + 0.040*"balsamic vinegar" + '
  '0.039*"plum tomatoes" + 0.039*"garlic cloves" + 0.037*"dijon mustard" + '
  '0.036*"fresh basil leaves"'),
 (5,
  '0.086*"carrots" + 0.081*"onions" + 0.054*"bacon" + 0.047*"bay leaf" + '
  '0.046*"water" + 0.038*"bay leaves" + 0.038*"celery ribs" + 0.035*"garlic '
  'cloves" + 0.032*"black peppercorns" + 0.030*"chopped parsley"'),
 (2,
  '0.115*"cayenne" + 0.114*"ground ginger" + 0.091*"cider vinegar" + '
  '0.073*"cream" + 0.034*"ginger paste" + 0.033*"ground cayenne pepper" + '
  '0.029*"chili paste" + 0.026*"turnips" + 0.021*"mace" + 0.020*"sugar"'),
 (26,
  '0.134*"heavy cream" + 0.107*"unsalted butter" + 0.049*"egg whites" + '
  '0.041*"egg yolks" + 0.041*"sugar" + 0.033*"whole milk" + 0.032*"grated '
  'nutmeg" + 0.031*"unsweetened cocoa powder" + 0.030*"half & half" + '
  '0.023*"heavy whipping cream"'),
 (25,
  '0.11

---

8. Попытайтесь интерпретировать сформированные темы. Например, сопоставьте темы с номерами 0, 1, 4 и 5.

In [15]:
# try to interpret the formed themes. For example, match topics numbered 0, 1, 4, and 5

pprint(lda.show_topic(0))
print()
pprint(lda.show_topic(1))
print()
pprint(lda.show_topic(4))
print()
pprint(lda.show_topic(5))

[('parmesan cheese', 0.119869),
 ('parmigiano reggiano cheese', 0.09320222),
 ('green beans', 0.080807075),
 ('asparagus', 0.05722176),
 ('arugula', 0.03724162),
 ('cream cheese, soften', 0.035477277),
 ('sunflower oil', 0.02303004),
 ('chorizo sausage', 0.022668082),
 ('lump crab meat', 0.02197175),
 ('cream style corn', 0.02077949)]

[('ground cardamom', 0.05830832),
 ('minced onion', 0.0489282),
 ('chili', 0.04595473),
 ('coriander', 0.03991124),
 ('pork shoulder', 0.036398746),
 ('kimchi', 0.03432069),
 ('extra firm tofu', 0.032082874),
 ('garlic chili sauce', 0.030227093),
 ('button mushrooms', 0.028017974),
 ('pineapple', 0.025830777)]

[('fish sauce', 0.0723343),
 ('garlic', 0.03661294),
 ('beansprouts', 0.027121866),
 ('carrots', 0.02664887),
 ('sugar', 0.026334979),
 ('soy sauce', 0.025600167),
 ('vegetable oil', 0.025268387),
 ('peanuts', 0.024368132),
 ('coconut milk', 0.02397205),
 ('red chili peppers', 0.023736114)]

[('carrots', 0.08567586),
 ('onions', 0.080984674),
 ('b

---

9. Для нескольких слов определите наиболее релевантные этим словам темы (метод `get_term_topics()`).

In [16]:
# for several words, determine the most relevant topics for these words (method get_term_topics())

pprint(lda.get_term_topics('salt'))
print()
pprint(lda.get_term_topics('pepper'))

[(3, 0.05424403),
 (5, 0.029494204),
 (6, 0.054164577),
 (7, 0.062731065),
 (9, 0.0723171),
 (10, 0.05393841),
 (11, 0.02680518),
 (12, 0.09409463),
 (15, 0.015481716),
 (17, 0.020700226),
 (19, 0.024033727),
 (20, 0.022147462),
 (22, 0.06355096),
 (24, 0.084869176),
 (25, 0.10594376),
 (28, 0.03657019),
 (29, 0.01912095),
 (30, 0.047216136),
 (31, 0.06914508),
 (33, 0.049788702),
 (35, 0.056804717),
 (36, 0.032120094),
 (37, 0.053656235),
 (38, 0.063322),
 (39, 0.053992502)]

[(3, 0.026661782),
 (6, 0.03329465),
 (7, 0.012246904),
 (12, 0.012200584),
 (25, 0.10978432),
 (28, 0.0103052),
 (31, 0.021752827),
 (33, 0.028655412),
 (37, 0.030004803),
 (38, 0.012973453),
 (39, 0.012258683)]


---

10. Оцените качество полученной модели, вычислив когерентность токенов и перплексию.

In [17]:
# assess the quality of the resulting model by calculating token coherence and perplexity.

coherence_model_lda = CoherenceModel(model=lda, texts=[recipe['ingredients'] for recipe in recipes], dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

print(lda.log_perplexity(corpus))

Coherence Score:  0.4812033347218689
-6.786160310823578


* about coherence score: https://stackoverflow.com/questions/60246570/gensim-lda-coherence-score-nan, which refers to the degree to which words or tokens in a sequence are related to each other in a semantically meaningful way. In other words, it is a measure of how well the words in a text fit together to create a coherent overall meaning.
* the negative sign of `log_perplexity` is just because it's a logarithm of a number. perplexity is basically the generative probability of that sample (or chunk of sample), it should be as high as possible. since log(x) is monotonically increasing with x, gensim perplexity should also be high for a good model. so in our case, for example, "-6" is better than "-7". and to understand the meaning of perplexity in this context, it is a measure of how well a language model is able to predict the next word in a sequence based on the words that came before it. It is calculated as the exponentiated average log-probability of the language model. A lower perplexity indicates that the language model is better at predicting the next word in a sequence.